# Camera Pose to Homography Conversion

This Jupyter notebook demonstrates how to convert camera pose information, defined in a world coordinate system, into a homography mapping. The mapping relates a defined plane in the world coordinate system to the camera plane. We begin by explaining how to define a homography mapping for a plane aligned with the world coordinate system. Then, we address a more general case where the world coordinate frame and the plane of interest are not aligned. The motivation for the second case is as follows: imagine you need to map points between the camera plane and a plane defined in the world coordinate frame. However, this plane may not necessarily align with the world coordinate system. Such a mapping enables determining the 3D coordinates, within the plane, of a point observed in the camera.

## Homography Between a Plane and the Camera

In this case the plane $\pi$ (the plane of interest) aligns with the XY-plane of the world coordinate system. Points in the image and the scene are related by a projective transformation. In Figure 1. $C$ is the camera centre, $x$ is the projection of the point $x_\pi$ onto the camera plane. The point $x_\pi$ lies on the XY-plane.

<figure align="center">
    <img src="./images/camera_pose_to_homography.png" width="400">
    <figcaption>Figure 1: Projective transformation from camera plane to a plane that aligns with the world coordinate frame.</figcaption>
</figure>

The projective transformation $x=PX$ is a map from the world coordinate frame to a point in the image coordinate frame. We have placed the world coordinate frame so that the XY-plane aligns with the plane $\pi$, meaning that the Z-coordinate is 0. Therefore we have

$$
x=PX=\begin{bmatrix}p_1 & p_2 & p_3 & p_4 \end{bmatrix} \begin{bmatrix}X \\ Y \\ 0 \\ 1 \end{bmatrix} = \begin{bmatrix}p_1 & p_2 & p_4 \end{bmatrix} \begin{bmatrix}X \\ Y \\ 1 \end{bmatrix}
$$

, and since the projection matrix $P$ of a calibrated camera is as follows

$$
P = K\begin{bmatrix}R & t\end{bmatrix} = K\begin{bmatrix}r_1 & r_2 & r_3 & t \end{bmatrix}
$$

Putting this information together, we have

$$
x = K\begin{bmatrix}r_1 & r_2 & r_3 & t \end{bmatrix} \begin{bmatrix}X \\ Y \\ 0 \\ 1 \end{bmatrix} = K\begin{bmatrix}r_1 & r_2 & t \end{bmatrix} \begin{bmatrix}X \\ Y \\ 1 \end {bmatrix}
$$

Since plane to plane mapping is a homography, we have the following

$$
H = K\begin{bmatrix}r_1 & r_2 & t \end{bmatrix}
$$

## Homography Between a Plane Defined in the World Coordinate Frame and the Camera

In this case we want to find the homography between any plane defined in the world coordinate frame and the camera plane. We have the following information:

* Camera calibration matrix: $K$
  * We have either calibrated the camera or obtained the camera calibration matrix using other means.
* Camera pose in the world coordinate system: $^cT_w$
  * We know the camera pose in the world coordinate system.
* Plane parameters: $\pi = \begin{bmatrix}\pi_1 & \pi_2 & \pi_3 & \pi_4\end{bmatrix}$
  * We have a known plane defined in the world coordinate frame

We expect that the direction vector $e_{w3}=\begin{bmatrix}0 & 0 & 1\end{bmatrix}$ of the world coordinate frame intersects with the plane $\pi$. In Figure 2. we have the following, $O_c$ is the camera centre, $O_w$ is the centre of the world coordinate frame and $O_\pi$ is the centre of the plane $\pi$.

<figure align="center">
    <img src="./images/camera_pose_to_homography_generic.jpg" width="500">
    <figcaption>Figure 2: Projective transformation from the camera to a plane that does not align with XY-plane of the world coordinate frame.
    </figcaption>
</figure>

We define the transformation $\left(^cT_w\right)^{-1} \left({}^{\pi}T_w\right) \left(^cT_{\pi}\right)$ so that it maps points defined in the camera coordinate frame back to the camera coordinate frame, or more formally

$$
x_c^{\prime} = \left( \left(^cT_w\right)^{-1} \left({}^{\pi}T_w\right) \left(^cT_{\pi}\right) \right) x_c = x_c
$$

The transformations are as follows:

* $^cT_w$ transformation matrix from the world coordinate frame to the camera coordinate frame
* ${}^{\pi}T_w$ transformation matrix from the world coordinate frame to the coordinate frame defined in the plane $\pi$
* $^cT_{\pi}$ transformation matrix from the coordinate frame defined in the plane $\pi$ to the camera coordinate frame

The transformation ${}^{\pi}T_w$ is defined as follows

$$
{}^{\pi}T_w = 
\begin{bmatrix}
{}^{\pi}R_w & {}^{\pi}t_w \\
0 & 1
\end{bmatrix}
$$

Next steps are defining both the location and the orientation of the plane coordinate frame in the world coordinate frame.

### Origin of the Plane $\pi$

The plane $\pi$ is defined if the world coordinate frame using the plane equation that is define later on. In order for us to define a coordinate system on the plane itself, we need to define where the origin of the plane is. In order to simplify things, we place origin of the plane coordinate frame at the location where the Z-axis of the world coordinate frame, defined by $e_{w3}=\begin{bmatrix}0 & 0 & 1\end{bmatrix}$, intersects with the plane. Therefore, we parameterize a line in the direction of the coordinate vector $e_z=\begin{bmatrix}0 & 0 & 1\end{bmatrix}^t$ with respect to $t$ as follows

$$
\begin{bmatrix}X \\ Y \\ Z\end{bmatrix} = O_w + e_z t = \begin{bmatrix}o_x \\ o_y \\ o_z\end{bmatrix} + \begin{bmatrix}0 \\ 0 \\ 1\end{bmatrix} t = \begin{bmatrix}0 \\ 0 \\ 1\end{bmatrix} t
$$

, where $O_w$ is the origin of the world coordinate system. The plane equation is as follows:

$$
\pi_1 X + \pi_2 Y + \pi_3 Z + \pi_4 = 0
$$

Next we plug in the coordinates from the parameterized line equation and we get

$$
t = -\dfrac{\pi_4}{\pi_3}
$$

$t$ is the length of the vector in the direction of $e_z$, and that is where the we place the origin of the plane $\pi$.

### Orientation of the Coordinate System on the Plane $\pi$

We want to align orientation of the plane coordinate system so that the Z-axis is aligned with the normal $n$ of the plane, while keeping X- and Y- directions aligned with the world coordinate system. Normal $n$ of a plane

$$
\pi = \begin{bmatrix}\pi_1 & \pi_2 & \pi_3 & \pi_4\end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}
$$

is

$$
n = \begin{bmatrix}\pi_1 & \pi_2 & \pi_3\end{bmatrix}
$$

Basis vectors of the world coordinate system are

$$
e_{w1} = \begin{bmatrix}1\\0\\0\end{bmatrix},\quad e_{w2} = \begin{bmatrix}0\\1\\0\end{bmatrix},\quad e_{w3} = \begin{bmatrix}0\\0\\1\end{bmatrix}
$$

We define the basis vectors $e_{p1}$, $e_{p2}$ and $e_{p3}$ of the plane coordinate system using the plane normal vector $n$ and the basis vector $e_{w2}$, so 
that the basis vector $e_{p3}$ it aligns with $n$

$$
e_{p3} = \dfrac{n}{||n||}
$$

Then we define the basis vector $e_{p1}$ to be perpendicular to $e_{w2}$ and $e_{p3}$

$$
e_{p1} = \dfrac{e_{w2} \times n}{||e_{w2} \times n||}
$$

, and finally the basis vector $e_{p2}$ is perpendicular to both $ep_1$ and $e_{p3}$

$$
e_{p2} = \dfrac{e_{p3} \times e_{p1}}{||e_{p3} \times e_{p1}||}
$$

Thus, orientation of the plane coordinate system is

$$
{}^{\pi}R_w = \begin{bmatrix}e_{p1} && e_{p2} && e_{p3}\end{bmatrix}
$$

### Combined Transformations

The transformation from the plane $\pi$ to the camera coordinate frame is defined as follows

$$
{}^{c}T_{\pi} = 
\begin{bmatrix}
{}^{c}R_{\pi} & {}^{c}t_{\pi} \\
0 & 1
\end{bmatrix}
$$

, where the rotation matrix and the translation vector are defined as follows

$$
\begin{aligned}
{}^{c}R_{\pi} &= {}^{c}R_{w} \left({}^{\pi}R_{w} \right)^{-1} \\
{}^{c}t_{\pi} &= {}^{c}t_{w} + {}^{c}R_{w} {}^{w}t_{\pi}
\end{aligned}
$$

, and

$$
{}^{w}t_{\pi} = - \left({}^{\pi}R_{w} \right)^{-1} {}^{\pi}t_{w}
$$

### Homography

When we calculate the homography

$$
H = K\begin{bmatrix}r_1 & r_2 & t \end{bmatrix}
$$

$r_1$ and $r_2$ are from ${}^{c}R_{\pi}$ and $t$ is ${}^{c}t_{\pi}$.

# Testing

In the following we create a test case for the homography mapping. World coordinate system origin is at $\left(0,0,0\right)$.

In [ ]:
from python_camera_library.utils import rotation
import numpy as np
import math
import open3d as o3d

### Plane

The equation of a plane with normal vector $n=(a,b,c)$ through the point $x_0=(x_0, y_0, z_0)$ is $n(x-x_0) = 0$. This gives the equation of a plane $ax + by + cz + d = 0$, where $d=-ax_0-by_0-cz_0$

In [ ]:
# Plane normal
n = np.array([0, 0, 1]).reshape(3, 1)
R = rotation(rot_x=0, rot_y=math.radians(10), rot_z=0)
n = R@n

# Plane origin
O_p = np.array([0, 0, -2]).reshape(3,1)
d = np.sum(-n*O_p)

# Plane equation
plane_equation = np.append(n, d)

print(f"Plane normal vector:\n{n}")
print(f"Plane origin:\n{O_p}")
print(f"Plane d-parameter:\n{d}")
print(f"Plane equation:\n{plane_equation}")

In [ ]:
# Intersection of the world coordinate system with the plane
t = -plane_equation[-1]/plane_equation[-2]
intersection = np.array([0., 0., 1.]).reshape(3,1) * t
print(f"Intersection of the world coordinate frame with the plane:\n{intersection}")

# Orientation of the plane coordinate system
ew_1 = np.array([1., 0., 0.])
ew_2 = np.array([0., 1., 0.])

ep_3 = n/np.linalg.norm(n)
ep_1 = np.cross(ew_2.flatten(), ep_3.flatten())
ep_1 /= np.linalg.norm(ep_1)
ep_2 = np.cross(ep_3.flatten(), ew_1.flatten())
ep_2 /= np.linalg.norm(ep_1)

# R_w2p and t_w2p transform points from the world to plane coordinate frame
R_w2p = np.hstack((ep_1.reshape(3,1), ep_2.reshape(3,1), ep_3.reshape(3,1))).T
t_w2p = -R_w2p.T @ intersection.copy()

print(f"R_w2p:\n{R_w2p}")
print(f"t_w2p:\n{t_w2p}")

# R_p2w and t_p2w transform points from the plane to world coordinate frame
t_p2w = -R_w2p.T @ t_w2p
R_p2w = R_w2p.T
print(f"R_p2w:\n{R_p2w}")
print(f"t_p2w:\n{t_p2w}")

### Test Transformations

Test the world to plane to camera transformations:
1. First transfer the point of interest from world to plane and then from plane to camera coordinate frame
2. Calculate the combined transformation world -> camera

Both transformations should produce same results.

In [ ]:
# Point of interest in the world coordinate frame
poi_w = np.array([0., 0., 0.]).reshape(3, 1)

# Point of interest in the plane coordinate frame
poi_p = R_w2p @ poi_w + t_w2p

print(f"poi_w:\n{poi_w}")
print(f"poi_p:\n{poi_p}")

# R_w2c and t_w2c transform coordinates from the world coordinate frame to the camera coordinate frame
R_w2c = np.array([[0., 1., 0.], [0., 0., -1.], [-1., 0., 0.]]).reshape(3,3)
t_w2c = np.array([0., 0., 3.]).reshape(3,1)
print(f"R_w2c:\n{R_w2c}")
print(f"t_w2c:\n{t_w2c}")

# R_p2c and t_p2c transform coordinates from the plane coordinate frame to the camera coordinate frame
R_p2c = R_w2c @ R_w2p.T
t_p2c = t_w2c - R_w2c @ R_w2p.T @ t_w2p

# Transfer the poi_w to the camera coordinate frame
poi_c1 = R_w2c @ poi_w + t_w2c

# Transfer the poi_p to the camera coordinate frame (poi_p is the same as poi_w but in the plane doordinate system)
poi_c2 = R_p2c @ poi_p + t_p2c

# poi_c1 and poi_c2 should have exactly the same values
print(f"poi_w -> poi_c1: {poi_w.flatten()} -> {poi_c1.flatten()}")
print(f"poi_p -> poi_c2: {poi_p.flatten()} -> {poi_c2.flatten()}")

# assert that they actually have the same values using assert_allclose
np.testing.assert_allclose(poi_c1, poi_c2, rtol=1e-5, atol=1e-15)

### Test Projection Using Camera Calibration Matrix K and Homography

We project the point of interest directly using the camera calibration matrix K and the homography matrix H. Both should produce the same results. Homography is defined as follows:

$$
H = K \begin{bmatrix}r_1 & r_2 & t_{p2c}\end{bmatrix}
$$

, where

$$
R_{p2c} = \begin{bmatrix}r_1 & r_2 & r_3\end{bmatrix}
$$

In [ ]:
# Point of interest defined in the plane coordinate frame
poi2_p = np.array([1., 0.5, 0.]).reshape(3,1)
print(f"Point of interest poi2_p in the plane coordinate frame:\n{poi2_p}")

# Transform poi2_p to the camera coordinate frame
poi2_c = R_p2c @ poi2_p + t_p2c

# Camera matrix
K = np.array([[100., 0., 0.], [0., 100., 0.], [0., 0., 1.]]).reshape(3,3)

# Homography from plane to the camera
H = K @ np.hstack((R_p2c[:, :2].reshape(3, 2), t_p2c))

# Project the point to the camera using camera calibration matrix K
uv1 = K @ poi2_c
uv1 /= uv1[-1, -1]
print(f"poi2_c projected to the camera using camera matrix K, uv1:\n{uv1}")

# Project the point to the camera using homography H
poi2_p_homog = poi2_p[:2].copy()
poi2_p_homog = np.append(poi2_p_homog, 1.).reshape(3,1)
uv2 = H @ poi2_p_homog
uv2 /= uv2[-1, -1]
print(f"poi2_p projected to the camera using homography H, uv2:\n{uv2}")

# Assert that uv1 and uv2 are equal (close)
np.testing.assert_allclose(uv1, uv2, rtol=1e-5, atol=1e-15)

# Backproject the point to the plane, this should have the same XY-coordinates as poi2_p
uv_backprojected = np.linalg.inv(H)@uv2
uv_backprojected /= uv_backprojected[-1, -1]
print(f"uv2 projected to the plane using inverse of homography H:\n{uv_backprojected}")

# Assert that the XY-coordinates are close
np.testing.assert_allclose(poi2_p[:2], uv_backprojected[:2], rtol=1e-5, atol=1e-10)

### Intersection Between a Camera Unit Vector and the Plane

Here we calculate where the back projection of the $uv1$, in the camera coordinate frame, intersects with the plane. Since $uv1$ is projection of $poi2$, these should both produce the same point.

In [ ]:
# Vector in the direction of uv1 in the camera coordinate frame
vect_c = np.linalg.inv(K) @ uv1
vect_c /= np.linalg.norm(vect_c)

R_c2w = R_w2c.T
t_c2w = -R_w2c.T @ t_w2c

# Vector in the direction of uv1 in the world coordinate frame
vect_w = R_c2w @ vect_c
vect_w /= np.linalg.norm(vect_w)

# From the plane equation
d = plane_equation[-1]
n = plane_equation[:3]
length_vect = -(d+np.sum((n.flatten()*t_c2w.flatten()))) / (np.sum(n.flatten()*vect_w.flatten()))
plane_intersection = t_c2w + length_vect*vect_w
print(f"Length of the vector: {length_vect}")
print(f"Intersection:\n{t_c2w + length_vect*vect_w}")

# Convert poi2_p to world coordinates
poi2_w = R_p2w @ poi2_p + t_p2w
print(f"poi2_p:\n{poi2_p}")
print(f"poi2_p in world coordinate frame, poi2_w:\n{poi2_w}")

In [ ]:
# Function to create a plane mesh with rotation and translation, visible from both sides
def create_plane(center, normal, width, height, R, t):
    # Calculate two orthogonal vectors in the plane
    if np.allclose(normal, [0, 0, 1]):
        v1 = np.array([1, 0, 0])
    else:
        v1 = np.cross(normal, [0, 0, 1])
        v1 /= np.linalg.norm(v1)

    v2 = np.cross(normal, v1)
    v2 /= np.linalg.norm(v2)

    # Calculate the four corners of the plane
    half_width = width / 2
    half_height = height / 2
    corners = np.array([
        center + half_width * v1 + half_height * v2,
        center - half_width * v1 + half_height * v2,
        center - half_width * v1 - half_height * v2,
        center + half_width * v1 - half_height * v2,
    ])

    # Apply rotation and translation to the corners
    corners = (R @ corners.T).T + t.reshape(1, 3)

    # Create the mesh
    vertices = o3d.utility.Vector3dVector(corners)
    triangles = o3d.utility.Vector3iVector([[0, 1, 2], [2, 3, 0]])
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = vertices
    mesh.triangles = triangles
    mesh.compute_vertex_normals()

    # Add colors to the mesh for better visualization
    mesh.paint_uniform_color([0.7, 0.7, 0.7])  # Light gray color

    return mesh

# Function to create a sphere
def create_sphere(position, radius, color):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    sphere.compute_vertex_normals()
    sphere.paint_uniform_color(color)
    sphere.translate(position)
    return sphere

In [ ]:
# Create visualization for the world coordinate frame
world_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

# Create visualization for the plane coordinate frame
plane_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=O_p)
plane_coordinate_frame.rotate(R, center=O_p)

# Create visualization for the camera coordinate frame
camera_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=t_c2w)
camera_coordinate_frame.rotate(R_c2w, center=t_c2w)

# Create the plane
plane = create_plane(center=O_p.flatten(), normal=n, width=2., height=2., R=np.eye(3), t=np.array([0,0,0]))

# Shows POIs -> these should overlap, we should see only the red ball
poi = create_sphere(plane_intersection, 0.02, [0.0, 1.0, 0.0])
poi2 = create_sphere(poi2_w, 0.02, [1.0, 0.0, 0.0])

# Draw everything
o3d.visualization.draw_geometries([world_coordinate_frame, plane, plane_coordinate_frame, camera_coordinate_frame, poi, poi2])

print("Red is the X-axis")
print("Green is the Y-axis")
print("Blue is the Z-axis")